In [1]:
# pip install opencv-python

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [15]:
def extract_orb_features(image_path):
    """
    Extracts ORB features from an image.
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")
    # Initialize the ORB detector
    orb = cv2.ORB_create()
    # Find the keypoints and descriptors with ORB
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

def visualize_keypoints(image_path):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("The image path is invalid.")

    # Initialize the AKAZE detector
    ORB = cv2.ORB_create()

    # Detect keypoints and descriptors with AKAZE
    kp, des = ORB.detectAndCompute(img, None)
    kp = kp[:100]
    len(kp)
    # Draw keypoints on the image
    img_with_keypoints = cv2.drawKeypoints(img, kp, None, color=(255, 0, 0), flags=cv2.DrawMatchesFlags_DEFAULT)

    # Display the image with keypoints
    plt.imshow(img_with_keypoints, cmap='gray')
    plt.title(f'Image: {image_name(image_path)} Keypoints')
    plt.axis('off')
    plt.show()

def image_name(image_path):
    return os.path.splitext(os.path.basename(image_path))[0]


image1_path = 'nk_collection_meubels_cleaned/meubel_1.jpg'
image2_path = 'nk_collection_meubels_cleaned/meubel_2.jpg'

import pandas as pd

def get_table(sims):
    """
    This function takes the output produced by either the compute_similarities \ 
    or compute_similarities_testsets function, and returns a pandas dataframe/table \
    and also saves it in excel.
    """
    
    data = {}
    rows = []

    for key, value in sims.items():
        if key[0] not in data:
            data[key[0]] = []
        if key[1] not in rows:
            rows.append(key[1])
        data[key[0]].append(value)
        
    data = {key[:key.rfind(".")]:value for key, value in data.items()}
    rows = [row[:row.rfind(".")] for row in rows]
        
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T
    
def get_table1(sims):
    """
    This function takes the output produced by either the compute_similarities 
    or compute_similarities_testsets function, and returns a pandas dataframe/table 
    and also saves it in excel.
    """
    
    data = {}
    rows = []
    
    mapping = {
        "kast_nk.jpg": "closet_nk.jpg",
        "speeltafel_nk.png": "card_table_nk.png",
        "tafel_nk.jpg": "table_nk.jpg",
        "dressoir_nk.jpg": "dresser_nk.jpg",
        "stoel_nk.jpg": "chair_nk.jpg",
        "kast_mccp.jpg": "closet_mccp.jpg",
        "speeltafel_mccp.png": "card_table_mccp.png",
        "tafel_mccp.jpg": "table_mccp.jpg",
        "dressoir_mccp.jpg": "dresser_mccp.jpg",
        "stoel_mccp.jpg": "chair_mccp.jpg",
    }

    for (key1, key2), value in sims.items():
        file1 = os.path.basename(key1)
        file2 = os.path.basename(key2)
        
        if mapping[file1] not in data:
            data[mapping[file1]] = []
        if mapping[file2] not in rows:
            rows.append(mapping[file2])
        
        value = np.round(value, 3)
        data[mapping[file1]].append(value)
        
    data = {key[:key.rfind(".")]: value for key, value in sorted(data.items())}
    rows = [row[:row.rfind(".")] for row in rows]
    rows_indices_begin = list(range(len(rows)))
    data_values = list(data.values())
    rows_indices_end = sorted(rows_indices_begin, key=lambda i: rows[i])
    rows = sorted(rows)
    
    for key, value in data.items():
        new_value = []
        for i in rows_indices_end:
            new_value.append(value[i])
            
        data[key] = new_value
    
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T

<>:46: SyntaxWarning: invalid escape sequence '\ '
<>:46: SyntaxWarning: invalid escape sequence '\ '
C:\Users\used\AppData\Local\Temp\ipykernel_10592\2330247682.py:46: SyntaxWarning: invalid escape sequence '\ '
  """


In [16]:
def orb_similarity(img1, img2):
    """
    Calculating the similarity between two images using ORB
    """
    orb = cv2.ORB_create(nfeatures=100,scaleFactor=1.1,nlevels=12,WTA_K=4, 
                          scoreType=cv2.ORB_HARRIS_SCORE, patchSize=21, fastThreshold=10)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    if des1 is None or des2 is None:
        return 0.0

    #Use BFMatcher with default params
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Apply RANSAC to find the best transformation
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 15)
    matches_mask = mask.ravel().tolist()

    num_inliers = np.sum(matches_mask)
    num_matches = len(matches)
    
    similarity_percentage = (num_inliers / num_matches) * 100
    return similarity_percentage

def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_testset", 
                                  nk_path="nk_testset"):
    """
    Computes similarities between images from Munich and NK test sets using ORB features.
    """
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        try:
            nk_img = cv2.imread(nk_img_path, cv2.IMREAD_GRAYSCALE)
            if nk_img is None:
                raise ValueError(f"Image at {nk_img_path} could not be loaded.")
        except ValueError as e:
            print(e)
            continue
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            try:
                munich_img = cv2.imread(munich_img_path, cv2.IMREAD_GRAYSCALE)
                if munich_img is None:
                    raise ValueError(f"Image at {munich_img_path} could not be loaded.")
            except ValueError as e:
                print(e)
                continue
            similarity = orb_similarity(nk_img, munich_img)
            similarities[(nk_img_path, munich_img_path)] = similarity
        
    return similarities

# Usage
nk_path = "nk_testset"
munich_path = "munich_testset"

if not os.path.exists(nk_path):
    raise FileNotFoundError(f"The path {nk_path} does not exist.")
if not os.path.exists(munich_path):
    raise FileNotFoundError(f"The path {munich_path} does not exist.")

nk_testset = os.listdir(nk_path)
munich_testset = os.listdir(munich_path)

sims = compute_similarities_testsets(munich_testset, nk_testset, munich_path, nk_path)

get_table1(sims)




,card_table_mccp,chair_mccp,closet_mccp,dresser_mccp,table_mccp
card_table_nk,45.455,62.500,45.455,54.545,64.286
chair_nk,50.000,42.105,42.857,50.000,50.000
closet_nk,50.000,62.500,40.000,47.059,42.857
dresser_nk,40.000,40.000,42.857,56.250,50.000
table_nk,40.000,33.333,40.000,47.059,50.000


In [19]:
def compute_similarities(nk_img_path, munich_imgs, path):
    """
    This function takes three arguments: 
    - nk_img, which is a single image from the nk collection. 
    - munich_imgs, this contains all images from the Munich Database. 
    - path, this is the path to the gray scaled Munich Database.
    
    It then computes the feature descriptor for the nk collection image and all the images in the \
    Munich Database. Afterwards takes the dot-product to get the dot-product similiarity. It then \
    saves the similarity and the two images as key-value pairs in a dictionary. 
    """
    
    nk_img = cv2.imread(nk_img_path, cv2.IMREAD_GRAYSCALE)
    similarities = {}
    i = 0

    for img_name in munich_imgs:
        if i == 1000:
            break
        print(i)
        i += 1
        img_path = os.path.join(path, img_name)
        munich_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if nk_img is not None and munich_img is not None and i != 428:
            similarity = orb_similarity(nk_img, munich_img)
            nk_img_name = os.path.basename(nk_img_path)
            munich_img_name = os.path.basename(img_path)
            similarities[(nk_img_name, munich_img_name)] = similarity

    return similarities

# Example usage
nk_img = "test_dataset_gray/kast_nk.jpg"
munich_imgs = os.listdir("scraped_images_grayscaled_big")
path = "scraped_images_grayscaled_big"

similarities = compute_similarities(nk_img, munich_imgs, path)
print(similarities)
get_table(similarities)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,1245_9632_id=cp130764_linz,0719_4326_id=cp161541_badv,0571_2865-1_id=cp153185_badv,0101_562_id=cp168851_badv,2081_41379_id=cp159297_badv,0697_4244-1_id=cp160763_badv,0327_1595-5_id=cp143235_badv,0497_2394-34_id=cp150371_badv,1732_21836-28_id=cp147429_badv,0642_3848_id=cp132737_linz,...,0962_7380_id=cp171211_badv,0377_1689-1_id=cp144637_badv,0528_2441-7_id=cp151138_badv,1284_10047_id=cp133046_linz,0535_2452-5_id=cp151271_badv,1297_10341_id=cp135374_badv,2320_47992_id=cp166038_badv,0988_7611_id=cp171684_badv,0944_7051-26_id=cp170758_badv,0770_4714-43_id=cp165393_badv
kast_nk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def get_top_10_similarities(similarities):
    """
    Get the top 10 highest similarity values for each NK image from the similarities dictionary.
    
    Parameters:
    - similarities: A dictionary where keys are (NK image name, Munich image name) tuples
                    and values are similarity scores.
                    
    Returns:
    - A dictionary where keys are NK image names and values are lists of tuples
      (Munich image name, similarity) sorted by similarity in descending order.
    """
    top_10_similarities = {}
    
    for nk_img_name in set(key[0] for key in similarities.keys()):
        # Filter similarities for current NK image
        nk_similarities = [(munich_img_name, similarity) for (nk, munich_img_name), similarity in similarities.items() if nk == nk_img_name]
        
        # Sort by similarity in descending order
        sorted_similarities = sorted(nk_similarities, key=lambda x: x[1], reverse=True)
        
        # Get top 10 similarities
        top_10_similarities[nk_img_name] = sorted_similarities[:10]
    
    return top_10_similarities


get_top_10_similarities(similarities)


{'kast_nk.jpg': [('1180_8966_id=cp175276_badv.jpg', 90.0),
  ('1920_33193-41_id=cp155035_badv.jpg', 90.0),
  ('2029_37136_id=cp157551_badv.jpg', 85.71428571428571),
  ('1016_7899-32_id=cp172200_badv.jpg', 84.61538461538461),
  ('1075_7970-1_id=cp173391_badv.jpg', 80.0),
  ('2163_43961_id=cp162313_badv.jpg', 78.57142857142857),
  ('0086_420-89_id=cp160122_badv.jpg', 76.92307692307693),
  ('0705_4246-21_id=cp160866_badv.jpg', 73.33333333333333),
  ('2196_44726_id=cp163068_badv.jpg', 73.33333333333333),
  ('2149_43755_id=cp162005_badv.jpg', 72.72727272727273)]}